# 03 - DnD DM World-State Engine (Portable Example)

This notebook provides a reusable engine-style implementation for testing and later integration:
- persistent world state
- tool schemas for Ollama
- iterative tool-calling loop
- stop-hook style completion checks
- per-turn trace output


In [5]:
import json
import random
import re
import sys
from copy import deepcopy
from pathlib import Path
from typing import Any

# Ensure repo root is importable when running this notebook from its folder.
repo_root = Path.cwd()
while repo_root != repo_root.parent and not (repo_root / "orchestrator").exists():
    repo_root = repo_root.parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from orchestrator.llm_interaction.adapter import LLMAdapter

MODEL = "llama3.1:8b"


In [6]:
def build_initial_world_state() -> dict[str, Any]:
    return {
        "setting": "Ashen Coast",
        "location": {
            "name": "Port Ember",
            "description": "A storm-battered harbor city built on black volcanic stone.",
        },
        "party": {
            "name": "The Lantern Company",
            "members": ["Kara (Rogue)", "Brann (Cleric)", "Ilya (Wizard)"],
            "inventory": ["50 gp", "healing potion", "smoke bomb"],
        },
        "npcs": {},
        "quests": [],
        "log": [],
    }


def world_summary(state: dict[str, Any]) -> str:
    return json.dumps({
        "setting": state["setting"],
        "location": state["location"],
        "party": state["party"],
        "npcs": state["npcs"],
        "quests": state["quests"],
    }, indent=2)


In [7]:
class DungeonMasterEngine:
    def __init__(self, model: str = MODEL):
        self.model = model
        self.adapter = LLMAdapter(model=model)
        self.state = build_initial_world_state()
        self.system_prompt = (
            "You are a DnD dungeon master with strict state discipline. "
            "Use tools for any world mutation or state lookup. "
            "Never invent tool outputs. End each final response with 2-3 player choices."
        )
        self.history: list[dict[str, Any]] = [
            {
                "role": "user",
                "content": "Initial world state:" + chr(10) + world_summary(self.state),
            },
        ]

        self.tools = self._build_tools()
        self.tool_impl = {
            "read_world": self.read_world,
            "update_location": self.update_location,
            "create_npc": self.create_npc,
            "set_npc_attitude": self.set_npc_attitude,
            "add_quest": self.add_quest,
            "resolve_quest": self.resolve_quest,
            "roll_dice": self.roll_dice,
            "add_log_entry": self.add_log_entry,
        }

    def _build_tools(self) -> list[dict[str, Any]]:
        return [
            {
                "type": "function",
                "function": {
                    "name": "read_world",
                    "description": "Read the full world state or a specific top-level section.",
                    "parameters": {
                        "type": "object",
                        "properties": {"section": {"type": "string"}},
                        "required": [],
                    },
                },
            },
            {
                "type": "function",
                "function": {
                    "name": "update_location",
                    "description": "Set current location name and description.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "name": {"type": "string"},
                            "description": {"type": "string"}
                        },
                        "required": ["name", "description"],
                    },
                },
            },
            {
                "type": "function",
                "function": {
                    "name": "create_npc",
                    "description": "Create or replace an NPC.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "name": {"type": "string"},
                            "role": {"type": "string"},
                            "attitude": {"type": "string"}
                        },
                        "required": ["name", "role", "attitude"],
                    },
                },
            },
            {
                "type": "function",
                "function": {
                    "name": "set_npc_attitude",
                    "description": "Update an existing NPC attitude.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "name": {"type": "string"},
                            "attitude": {"type": "string"}
                        },
                        "required": ["name", "attitude"],
                    },
                },
            },
            {
                "type": "function",
                "function": {
                    "name": "add_quest",
                    "description": "Add a quest with open status.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "title": {"type": "string"},
                            "objective": {"type": "string"}
                        },
                        "required": ["title", "objective"],
                    },
                },
            },
            {
                "type": "function",
                "function": {
                    "name": "resolve_quest",
                    "description": "Mark quest as resolved with outcome text.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "title": {"type": "string"},
                            "outcome": {"type": "string"}
                        },
                        "required": ["title", "outcome"],
                    },
                },
            },
            {
                "type": "function",
                "function": {
                    "name": "roll_dice",
                    "description": "Roll NdM(+/-K), example 2d6+1.",
                    "parameters": {
                        "type": "object",
                        "properties": {"expression": {"type": "string"}},
                        "required": ["expression"],
                    },
                },
            },
            {
                "type": "function",
                "function": {
                    "name": "add_log_entry",
                    "description": "Append a structured note to campaign log.",
                    "parameters": {
                        "type": "object",
                        "properties": {"text": {"type": "string"}},
                        "required": ["text"],
                    },
                },
            },
        ]

    # ---- tool implementations ----
    def read_world(self, section: str | None = None) -> dict[str, Any]:
        if not section:
            return deepcopy(self.state)
        if section not in self.state:
            raise ValueError(f"Unknown section '{section}'")
        return deepcopy(self.state[section])

    def update_location(self, name: str, description: str) -> dict[str, Any]:
        self.state["location"] = {"name": name, "description": description}
        return self.state["location"]

    def create_npc(self, name: str, role: str, attitude: str) -> dict[str, Any]:
        self.state["npcs"][name] = {"role": role, "attitude": attitude}
        return {"name": name, **self.state['npcs'][name]}

    def set_npc_attitude(self, name: str, attitude: str) -> dict[str, Any]:
        if name not in self.state["npcs"]:
            raise ValueError(f"NPC '{name}' does not exist")
        self.state["npcs"][name]["attitude"] = attitude
        return {"name": name, **self.state['npcs'][name]}

    def add_quest(self, title: str, objective: str) -> dict[str, Any]:
        quest = {"title": title, "objective": objective, "status": "open"}
        self.state["quests"].append(quest)
        return quest

    def resolve_quest(self, title: str, outcome: str) -> dict[str, Any]:
        for q in self.state["quests"]:
            if q["title"] == title:
                q["status"] = "resolved"
                q["outcome"] = outcome
                return q
        raise ValueError(f"Quest '{title}' not found")

    def roll_dice(self, expression: str) -> dict[str, Any]:
        m = re.fullmatch(r"(\d+)d(\d+)([+-]\d+)?", expression.strip())
        if not m:
            raise ValueError(f"Invalid dice expression: {expression}")
        n, sides, mod = int(m.group(1)), int(m.group(2)), int(m.group(3) or 0)
        rolls = [random.randint(1, sides) for _ in range(n)]
        return {"expression": expression, "rolls": rolls, "modifier": mod, "total": sum(rolls) + mod}

    def add_log_entry(self, text: str) -> dict[str, Any]:
        entry = {"text": text}
        self.state["log"].append(entry)
        return entry

    # ---- model + loop helpers ----
    def _execute_tool(self, name: str, args: dict[str, Any]) -> dict[str, Any]:
        fn = self.tool_impl.get(name)
        if not fn:
            return {"ok": False, "error": f"Unknown tool: {name}"}
        try:
            return {"ok": True, "result": fn(**args)}
        except Exception as e:
            return {"ok": False, "error": str(e)}

    def _stop_hook(self, assistant_text: str, stop_hook_active: bool) -> str | None:
        text = (assistant_text or "").lower()
        has_choices = "what do you do" in text or "choices" in text or "options" in text
        if not has_choices:
            return "Provide 2-3 concrete player choices before completing the turn."

        if stop_hook_active and len(assistant_text.strip()) < 120:
            return "When stop hook is active, provide more detailed scene resolution."

        return None

    def run_turn(self, player_input: str, max_iterations: int = 12) -> dict[str, Any]:
        loop_messages = [
            *self.history,
            {"role": "user", "content": f"Player action: {player_input}"},
        ]

        loop = self.adapter.run_tool_loop(
            stage="notebook_world_engine",
            system_prompt=self.system_prompt,
            messages=loop_messages,
            tools=self.tools,
            tool_executor=self._execute_tool,
            max_iterations=max_iterations,
            stop_hook=lambda assistant_text, stop_hook_active: self._stop_hook(assistant_text, stop_hook_active),
        )

        self.history = loop["messages"]

        if loop["status"] == "completed":
            narration = loop["final_answer"]
            self.state["log"].append({"text": narration, "type": "narration"})
            return {
                "status": "completed",
                "narration": narration,
                "rounds": loop["rounds"],
            }

        return {
            "status": "max_iterations",
            "narration": "Stopped after max iterations.",
            "rounds": loop["rounds"],
        }


In [8]:
engine = DungeonMasterEngine()

turns = [
    "We ask around Port Ember for rumors about smugglers.",
    "I bribe the dock clerk and ask where the Midnight Lantern docks.",
    "We follow the lead to the warehouse district and prepare an ambush.",
]

for i, prompt in enumerate(turns, start=1):
    out = engine.run_turn(prompt)
    print(f"\n===== TURN {i} ({out['status']}) =====")
    print(out["narration"])
    print("tool rounds:", len(out["rounds"]))

print("\n===== WORLD STATE SNAPSHOT =====\n")
print(world_summary(engine.state))



===== TURN 1 (completed) =====
The party now has several options:

1. **Investigate the caves**: Head into the nearby caves to see if they can gather more information on the smuggling route.
2. **Question the locals further**: Try to press the locals for more information about the smuggling operation, or ask if they know anything else about it.
3. **Visit the local guild of adventurers**: See if any other adventurers in town have heard rumors or have leads on the smugglers.

Which option do you choose?
tool rounds: 4

===== TURN 2 (completed) =====
After bribing the dock clerk, Kara manages to extract some information from him. He leans in close and whispers that the Midnight Lantern docks are located on the far side of the harbor, near the old windmill. "Be careful," he warns, "the smugglers don't take kindly to strangers."

The party now has several options:

1. **Head to the Midnight Lantern docks**: Make your way to the dock and see if you can gather more information or even board